<a href="https://colab.research.google.com/github/Masum06/GenderDebiasing/blob/development2/Gender_Encrypter_Decrypter_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Latest

https://www.researchgate.net/publication/337918817_What_You_See_Is_What_You_Learn_Mitigating_Gender_Biases_from_Natural_Language_Processing_Context_through_Gender_Encrypted_Training 

# Gender Encryption

### Installation

In [1]:
import spacy
spacy.__version__

'2.2.4'

https://github.com/zalandoresearch/flair

https://github.com/huggingface/neuralcoref

In [ ]:
!pip uninstall torchvision
!pip install torchvision==0.4.0
!pip install flair==0.4.2
!pip uninstall spacy --yes
!pip install -U spacy==2.1.0
##!python -m spacy download en # SMALL VERSION OF SPACY, FAST DOWNLOAD
##!python -m spacy download en_core_web_md # MEDIUM VERSION OF SPACY
!python -m spacy download en_core_web_lg # LARGE VERSION OF SPACY, SLOW DOWNLOAD
!pip install neuralcoref --no-binary neuralcoref

Uninstalling torchvision-0.7.0+cu101:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/torchvision-0.7.0+cu101.dist-info/*
    /usr/local/lib/python3.6/dist-packages/torchvision/*
Proceed (y/n)? y
  Successfully uninstalled torchvision-0.7.0+cu101
     |████████████████████████████████| 8.8MB 2.7MB/s 
     |████████████████████████████████| 748.9MB 15kB/s 
  Found existing installation: torch 1.6.0+cu101
    Uninstalling torch-1.6.0+cu101:
      Successfully uninstalled torch-1.6.0+cu101
     |████████████████████████████████| 143kB 2.8MB/s 
     |████████████████████████████████| 133kB 8.7MB/s 
     |████████████████████████████████| 798kB 8.8MB/s 
     |████████████████████████████████| 1.1MB 16.8MB/s 
  Created wheel for sqlitedict: filename=sqlitedict-1.6.0-cp36-none-any.whl size=14689 sha256=b486d27b7caabc2793e2c1e2775401a312978fec34384780d441fe38fca3cdd1
  Stored in directory: /root/.cache/pip/wheels/bd/57/d3/907c3ee02d35e66f674ad0106e61f06eeeb98f6ee66a6cc3fe
  Created 

In [ ]:
import torch
torch.cuda.get_device_name(0)

'GeForce MX150'

In [ ]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


### Creating Language Resources

In [ ]:
male_pronouns   = ["he",  "him", "his$", "his", "himself"]
female_pronouns = ["she", "her", "her$", "hers", "herself"]
neutral_pronouns= ["zie", "zim", "zir", "zis", "zieself"]
merged_pronouns = ["he/she", "him/her", "his/her", "his/hers", "himself/herself"]

In [ ]:
gender_pronouns_dict = {}
gender_honorific_dict = {}

In [ ]:
for (g1,g2,g3,g4) in zip(male_pronouns, female_pronouns, neutral_pronouns, merged_pronouns):
    element = {"male": g1.replace("$",""), "female":g2.replace("$",""), "neutral":g3, "merged":g4}
    gender_pronouns_dict[g1] = gender_pronouns_dict[g2] = gender_pronouns_dict[g3] = gender_pronouns_dict[g4] =element

In [ ]:
male_hons   =  ["Mr.", "Mr", "Md.", "Md", "Sir", "Lord", "Mister"]
female_hons =  ["Ms.", "Ms", "Mst.", "Mst", "Madam", "Lady", "Miss"]
neutral_hons = ["Mx.", "Mx", "Mx.", "Mx", "Sir/Madam", "Lord/Lady", "Mister/Miss"]
married_hons = ["Mrs.", "Mrs", "Mst.", "Mst", "Madam", "Lady", "Mis'ess"]
merged_hons =  ["Mr./Ms.", "Mr/Ms", "Md./Mst.", "Md/Mst", "Sir/Madam", "Lord/Lady", "Mister/Miss"]

In [ ]:
for (h1,h2,h3,h4,h5) in zip(male_hons, female_hons, neutral_hons, married_hons, merged_hons):
    element = {"male": h1, "female":h2, "neutral":h3, "married_fem":h4, "merged":h5}
    gender_honorific_dict[h1] = gender_honorific_dict[h2] = gender_honorific_dict[h3] = \
    gender_honorific_dict[h4] = gender_honorific_dict[h5] = element

### Imports

In [ ]:
import re

In [ ]:
from flair.data import Sentence
from flair.models import SequenceTagger
#from segtok.segmenter import split_single

In [ ]:
import flair
assert flair.__version__=='0.4.2'

In [ ]:
from keras.models import load_model

Using TensorFlow backend.
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:

In [ ]:
import spacy
import neuralcoref

In [ ]:
import keras
keras.__version__

'2.2.4'

In [ ]:
import torch
torch.__version__

'1.3.0'

### Loading Essentials

In [ ]:
!git clone --single-branch --branch development2 https://github.com/Masum06/GenderDebiasing.git

Cloning into 'GenderDebiasing'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 147 (delta 10), reused 6 (delta 2), pack-reused 127
Receiving objects: 100% (147/147), 43.25 MiB | 9.31 MiB/s, done.
Resolving deltas: 100% (69/69), done.
Checking out files: 100% (47/47), done.


In [ ]:
cd GenderDebiasing

F:\GDrive RA\Thesis\GenderDebiasing2\GenderDebiasing


In [ ]:
cd ..

F:\GDrive RA\Thesis\GenderDebiasing2


In [ ]:
import json

char2idx = 'resources/char2idx.json'
idx2char = 'resources/idx2char.json'
with open(char2idx, 'r') as fp:
    char2idx = json.load(fp)
    
with open(idx2char, 'r') as fp:
    idx2char = json.load(fp)

In [ ]:
model_name = 'resources/char_rnn_hsc_model_0.h5'
model = load_model(model_name)

2020-06-08 19:21:29,071 From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

2020-06-08 19:21:29,128 From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

2020-06-08 19:21:29,150 From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

2020-06-08 19:21:29,324 From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

2020-06-08 19:21:29,332 From c:\users\hp\appdata\local\programs\python

In [ ]:
#nlp = spacy.load('en_core_web_lg') # en_core_web_sm #en_core_web_md
import en_core_web_lg
nlp = en_core_web_lg.load()
neuralcoref.add_to_pipe(nlp)

In [ ]:
tagger_ner = SequenceTagger.load('ner')
tagger_pos = SequenceTagger.load('pos')

2020-06-08 19:21:47,202 loading file C:\Users\Hp\.flair\models\en-ner-conll03-v0.4.pt
2020-06-08 19:22:01,368 loading file C:\Users\Hp\.flair\models\en-pos-ontonotes-v0.2.pt


### Necessary Methods

Name2Gender

In [ ]:
import numpy as np
from keras.preprocessing import sequence

# Converts a name into vector
def name2vectorTest(name):
    name = name.lower()
    new_name = ""
    for char in name:
      if char in char2idx:
        new_name += char
    chars = list(new_name)
    vector = [ char2idx[c] for c in chars ]
    return np.array(vector)

# Converts names to fixed size tensor
def names2tensorTest(names, maxlen=25):
    namelist = [name2vectorTest(name) for name in names]
    return sequence.pad_sequences(np.array(namelist), maxlen=maxlen)  # root of all troubles

def name2gender(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  if result:
    return "male"
  else:
    return "female"
  
def isMale(name):
  result = model.predict_classes(np.array(names2tensorTest([name.lower()])))[0][0]
  return result

Storing Names in Dictionary

In [ ]:
def store(name, name_found, Name2Key, Key2Name, num_keys): 
    
    if name_found not in Name2Key:
        #global num_keys
        num_keys+=1
        key = "PER_"+str(num_keys)
        #gender = name2gender(name_found)
        alias = None
        element = {"name": name, "key": key, "gender":None, "alias":alias, "is_alias": False, "alias_to": None}
        Name2Key[name_found] = element
        Key2Name[key] = {"name": name, "gender": None, "alias": alias}
    
    if name not in Name2Key:
        element_alias = {"name": name, "is_alias": True, "alias_to": name_found}
        Name2Key[name] = element_alias
        Name2Key[name_found]["alias"] = name
        
    return Name2Key[name_found]["key"], num_keys

# Unit Gender Encryption

In [ ]:
def gender_encrypt(s):
  s = ' '.join(s.split())
  doc = nlp(s)
  tokenized_text = ' '.join([token.text for token in doc])
  oracle = []
  coref2name = {}

  # POS TAG
  sent = Sentence(tokenized_text)
  tagger_ner.predict(sent)
  tagger_pos.predict(sent)
  tagged_list = sent.to_tagged_string().split()
  tokens_pos = []
  pos = []
  count = 0
  for i in range(0,len(tagged_list),2):
      tokens_pos.append(tagged_list[i])
      count = count+1
      pos.append(tagged_list[i+1])
      
      if tagged_list[i].lower() in male_pronouns+female_pronouns:
          oracle.append(2)
      elif tagged_list[i+1] in ['<B-PER/NNP>', '<I-PER/NNP>', '<E-PER/NNP>', '<S-PER/NNP>']:
          oracle.append(4)
      else:
          oracle.append(0)

  # COREFERENCE RESOLUTION 
  if len(doc)!=len(tokens_pos):
    tokens_doc = [token.text for token in doc]
    print("doc:", tokens_doc)
    print("pos:", tokens_pos)
    return None, None
  
  coref_stack = []
  name_stack = []
  for i in range(len(doc)):
      token = doc[i]
      if token._.in_coref:
          coref_stack.append(tokens_pos[i])
          if oracle[i] == 4:
              name_stack.append(tokens_pos[i])
          oracle[i] += 1
      else:
          if len(name_stack) > 0:
              name = ' '.join(name_stack)
              coref = ' '.join(coref_stack)
              #name2coref[name] = coref
              coref2name[coref] = name
              name_stack.clear()
          coref_stack.clear()

  # IF THE SENTENCE DOES NOT END WITH A PERIOD OR SPECIAL CHARACTER
  if len(name_stack) > 0:
      name = ' '.join(name_stack)
      name2coref[name] = ' '.join(coref_stack)
      name_stack.clear()
  coref_stack.clear()

  Name2Key = {}
  Key2Name = {}
  encrypted = []
  num_keys = 0
  i = 0
  while i<len(tokens_pos): 
      #print("Oracle ", i, tokens_pos[i])
      if oracle[i] == 2:
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          encrypted.append(gender_pronouns_dict[pronoun]["merged"])
      elif oracle[i] == 3:
          coref = doc[i]._.coref_clusters[0][0].text
          pronoun = tokens_pos[i].lower()
          if pos[i] == '<PRP$>':
              pronoun+="$"
          if coref in coref2name:
            name_found = coref2name[coref]
            key = Name2Key[name_found]["key"]
            if pronoun in male_pronouns:
              Name2Key[name_found]["gender"] = "male"
            elif pronoun in female_pronouns:
              Name2Key[name_found]["gender"] = "female"
            else:
              Name2Key[name_found]["gender"] = "<|unk|>"
            encrypted.append("<|coref|>")
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
            encrypted.append(key)
          else:
            encrypted.append(gender_pronouns_dict[pronoun]["merged"])
          
      elif oracle[i] in [4,5]:
          if i > 0 and tokens_pos[i-1] in gender_honorific_dict:
            hons = encrypted.pop()
            if hons in male_hons:
              gender = "male"
            elif hons in female_hons+married_hons:
              gender = "female"
            else:
              gender = "<|unk|>"
            encrypted.append("<|hons|>")
            encrypted.append(gender_honorific_dict[hons]["merged"])
          if pos[i] == '<S-PER/NNP>':
              name = tokens_pos[i]
          elif pos[i] == '<B-PER/NNP>':
              name = ""
              while True:
                  #print(i, oracle[i])
                  name += tokens_pos[i]
                  if pos[i] == '<E-PER/NNP>':
                      break
                  name += " "
                  i+=1
          
          if oracle[i] == 4:
              key, num_keys = store(name, name, Name2Key, Key2Name, num_keys)
              encrypted.append(key)
          else:
              coref = doc[i]._.coref_clusters[0][0].text
              name_found = coref2name[coref]
              if name == name_found:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append(key)
              else:
                  key, num_keys = store(name, name_found, Name2Key, Key2Name, num_keys)
                  encrypted.append("<|alias|>")
                  encrypted.append(key)
          Name2Key[key]["gender"] = gender
      else:
        encrypted.append(tokens_pos[i])
      i+=1
      
  encrypted_text = ' '.join(encrypted)

  return tokenized_text, Key2Name, encrypted_text

# Gender Decryption

In [ ]:
def gender_decrypt(encrypted_text, Key2Name):
  encrypted = encrypted_text.split()
  decrypted = []
  i = 0
  while i != len(encrypted):
    token = encrypted[i]
    if token in Key2Name:
      decrypted.append(Key2Name[token]["name"])
    elif token == '<|alias|>':
      i+=1
      key = encrypted[i]
      if Key2Name[key]["alias"]:
        decrypted.append(Key2Name[key]["alias"])
      else:
        decrypted.append(Key2Name[key]["name"])
    elif token == '<|coref|>':
      startOfSent = (i==0 or encrypted[i-1] == ".")
      pronoun =  encrypted[i+1]
      key = encrypted[i+2]
      gender = Key2Name[key]["gender"].lower()
      decrypted_pronoun = gender_pronouns_dict[pronoun][gender]
      decrypted_pronoun = decrypted_pronoun[0].upper()+decrypted_pronoun[1:] if startOfSent else decrypted_pronoun
      decrypted.append(decrypted_pronoun)
      i+=2
    elif token == "<|hons|>":
      hons = encrypted[i+1]
      key = encrypted[i+2] # NEED CHECK FOR HONS IN ALIAS
      gender = Key2Name[key]["gender"].lower()
      decrypted.append(gender_honorific_dict[hons][gender])
      i+=1
    elif token == "he/she":
        if i==0 or encrypted[i-1] == ".":
            decrypted.append("He/She")
        else:
            decrypted.append("he/she")
    else:
      decrypted.append(token)
    i+=1

  decrypted_text = ' '.join(decrypted)
  #decrypted_text = decrypted_text.replace(" .", ".")
  return decrypted_text

In [ ]:
key2name_changed = {'PER_1': {'name': 'Abira',
    'key': 'PER_1',
    'gender': 'Female',
    'alias': None,
    'is_alias': False,
    'alias_to': None},
  'PER_2': {'name': 'Shibir',
    'key': 'PER_2',
    'gender': 'Male',
    'alias': None,
    'is_alias': False,
    'alias_to': None}
  }

### Testing Encryption-Decryption

In [ ]:
s = "Mr. John is not a doctor. Ms. Katy is also not a nurse. John can fly, but she cannot swim."

In [ ]:
s = "Morshed Khan told newsmen that 'BNP is a massive party with huge popularity and public acceptance.' This party is now running its operations over Skype. He (Tarique) is operating from London over Skype and that too with only selected leaders. It has turned into the ‘Bangladesh Nationalist Skype Party’. This is painful. I believe the next generation must assume leadership within the party — that would be my recommendation.”"

In [ ]:
s = "Dineshwar then left behind his wife’s stabbed body and hanged himself from a tree in a field nearby."

In [ ]:
len(s.split())

18

In [ ]:
import time 
start = time.time()
text, key2name, encrypted = gender_encrypt(s)
end = time.time()

print(encrypted)
print(key2name)
print(end-start)

UnboundLocalError: local variable 'gender' referenced before assignment

In [ ]:
key2name = {'PER_1': {'name': 'Dineshwar', 'gender': 'merged', 'alias': None}}

In [ ]:
decrypted = gender_decrypt(encrypted, key2name)
decrypted

'Dineshwar then left behind his/her wife ’s stabbed body and hanged himself/herself from a tree in a field nearby .'

In [ ]:
decrypted == text

True

In [ ]:
en = gender_encrypt("Mr Masum Hasan is not a doctor. Masum is an engineer.")
en

NameError: name 'gender_encrypt' is not defined